In [576]:
# Python 3.8 (Kuzma's env: py_38)
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from itertools import chain
import qgrid

# Input Locations


In [577]:
root = r'C:\Users\samantha.kuzma\OneDrive - World Resources Institute'

# - - - FOLDERS - - -  - - -  - - - #
# ReservoirWatch specific
data_root = os.path.join(root, "ReservoirWatch", "Data")
raw_path = os.path.join(data_root, "Raw")
analysis_path = os.path.join(data_root, "Analysis")

context_gdb = os.path.join(analysis_path, "contextual.gdb")
cwc_path = os.path.join(raw_path, "CWC")

# Output Locations

In [578]:
'''
This file matches each Reservoir to its GRnD ID, CWC Name, State ID,
Hydrobasin 6 basins (contain reservoir, upstream, and downstream), and
FAO Major and Sub basins (contain reservoir, upstream, and downstream),
'''
res_ids_path = os.path.join(analysis_path, "reservoir_to_IDs_lookup.csv")
p4w_fao_path = os.path.join(analysis_path, "P4W_fao.shp")
p4w_hy6_path = os.path.join(analysis_path, "P4W_hy6.shp")

# Match reservoirs to spatial boundaries

### Read in spatial data (saved in Geodatabase)

In [531]:
# Read in 
res = gpd.read_file(context_gdb, layer='Reservoir')
# hy6 = gpd.read_file(context_gdb, layer='IND_aq30')
hy6 = gpd.read_file(context_gdb, layer='IND_hy6_wri')
ad2 = gpd.read_file(context_gdb, layer='IND_adm2_gadm36')
# fao = gpd.read_file(context_gdb, layer='IND_majbas_fao') # Excludes some downstream watersheds, can't use
fao = gpd.read_file(context_gdb, layer='ALL_majbas_fao') # Global FAO named basins

### Match CWC names to reservoir shapefile

In [532]:
# Match CWC data to shapefile/GRaND Reservoir IDs
res_lookup = res.filter(['GRAND_ID', 'DAM_NAME', 'RIVER', 'MAIN_BASIN'])
# Reservoir names
res_names = os.listdir(cwc_path) # CWC Reservoir Names
GRaND_names = res_lookup['DAM_NAME'].tolist() # GRaND Names (in shapefile)
# Find names that don't match
unmatched_names = sorted([x for x in GRaND_names if x not in res_names])
cwc_unmatched_names = sorted([x for x in res_names if x not in GRaND_names])
print("GRaND unmatched_names", unmatched_names)
print("CWC unmatched_names", cwc_unmatched_names)
# Create CWC Name column in shapefile
res_lookup['CWC_NAME'] = np.nan
# Add names that do match
res_lookup['CWC_NAME'][~res_lookup['DAM_NAME'].isin(unmatched_names)] = res['DAM_NAME'] # Add names that match
# Manually add the rest
res_lookup['CWC_NAME'][res_lookup['DAM_NAME']==' '] = 'Pench'
res_lookup['CWC_NAME'][res_lookup['DAM_NAME']=='Indirasagar'] = 'indirasagar'
res_lookup['CWC_NAME'][res_lookup['DAM_NAME']=='Panchet Hill'] = 'Panchet'
res_lookup['CWC_NAME'][res_lookup['DAM_NAME']=='Sriram Sagar'] = 'Sriramsagar'


GRaND unmatched_names [' ', 'Indirasagar', 'Panchet Hill', 'Sriram Sagar']
CWC unmatched_names ['Panchet', 'Pench', 'Sriramsagar', 'indirasagar']


<ipython-input-532-2b3974f0b115>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_lookup['CWC_NAME'][~res_lookup['DAM_NAME'].isin(unmatched_names)] = res['DAM_NAME'] # Add names that match


### Match CWC names to Hydrobasin 6, States, and FAO Basins

In [533]:
# Function to join reservoir shapefile to other geographical boundaries
def find_intersecting_ids(shape, shape_id):
    joined = gpd.sjoin(res, shape, how="left", op='intersects') # Join shapefile to reservoir shapefile
    list_ids = joined.groupby(['GRAND_ID'])[shape_id].agg(list).to_frame() # create a list of all IDs that intersect with reservoir
    list_ids[shape_id].apply(lambda x: list(set(x))) # Remove unique IDs
    return list_ids[shape_id]

# Create dataframe
res_ids = res_lookup.filter(['GRAND_ID', 'CWC_NAME']).set_index(['GRAND_ID'])
res_ids['PFAF_ID'] = find_intersecting_ids(hy6, 'PFAF_ID')
res_ids['HYBAS_ID'] = find_intersecting_ids(hy6, 'HYBAS_ID')
res_ids['GID_2'] = find_intersecting_ids(ad2, 'GID_2')
res_ids['MAJ_BAS'] = find_intersecting_ids(fao, 'MAJ_BAS')
res_ids['SUB_BAS'] = find_intersecting_ids(fao, 'SUB_BAS')



# Find Upstream and Downstream basins

In [534]:
# FUnction to ID upstream and downstream watersheds per basin
def basin_upstream_downstream(within_basins, lookp_basins, bas_ID, bas_DOWNSTREAM):
    # within_basins = list of basins that contain reservoirs
    # lookup_basins = original watershed data that links basins to their downstream basins
    # bas_ID = Uniqe basin ID used in original watershed data
    # bas_DOWNSTREAM = Field name used in original watershed data to list downstream basin
    
    # # Create dictionary to hold results of upstrea/downstream
    hyDict = {key: [] for key in within_basins} 
    # # Loop through every sub basin that contains reservoir
    for bas in within_basins:
#         print(bas)
    # - - - - - - - - - - - UPSTREAM - - - - - - - - - # 
#         print('Start Upstream')
        upstreams = [] # list to hold all upstream basins for selected sub_basin
        # 1st round, find all basins which flow to selected subbasins. IE upstream basinss
        ups = list(set(lookp_basins[bas_ID][lookp_basins[bas_DOWNSTREAM] == bas].tolist())) 
    #     # As long as new upstream basins are found, keep running the script
        while len(ups) > 0:
            # for every upstream basin in list, find it's upstream basins
            for up in ups:
                # First, add selected upstream basin to list of upstream basins
                upstreams.append(int(up)) # Add basin to list of upstream basins
                # Now find which basins are upstream of it
                ups_up = list(set(lookp_basins[bas_ID][lookp_basins[bas_DOWNSTREAM] == int(up)].tolist()))
                # Add those basins to the looping list of upstream basins
                ups = ups + ups_up
                # Remove the current selected upstream basin from looping list
                ups.remove(up)
                # Repeat For loop until no more upstream basins are found
        # Remove duplicates from upstream list
        upstreams = list(set(upstreams))  
        # Add to dictionary
        hyDict[bas].append(upstreams)
    # - - - - - - - - - - - DOWNSTREAM - - - - - - - - - #    
    #     # Find all downstream basins
#         print('Start Downstream')
        downstreams = [] # list to hold all downstream basins for selected sub_basin
        # 1st round, find all basins the selected subbasin flows to. IE downstream basins
        downs = list(set(lookp_basins[bas_DOWNSTREAM][lookp_basins[bas_ID] == bas].tolist())) 
        # Check to see if downstream exists
        if downs[0] == 0:
            print('check:', len(downs))
            downs = []
        # As long as new downstream basins are found, keep running the script
        while len(downs) > 0:
            # for every downstream basin in list, find it's downstream basins
            for down in downs:
                downstreams.append(int(down)) # Add basin to list of downstream basins
                # Now find which basins are downstream of it
                downs_down = list(set(lookp_basins[bas_DOWNSTREAM][lookp_basins[bas_ID] == down].tolist())) 
                # Add those basins to the looping list of downstream basins
                downs = downs + downs_down
                # Remove the current selected downstream basin from looping list
                downs.remove(down)
                # Repeat For loop until no more downstream basins are found
        # Remove duplicates from upstream list
        downstreams = list(set(downstreams))
        # Add to dictionary
        hyDict[bas].append(downstreams)

    # Turn results into dataframe
    df_hydro = pd.DataFrame.from_dict(hyDict).transpose()
    df_hydro.rename(columns = {0: "Upstream", 1: "Downstream"}, inplace = True)
    return df_hydro

## Find upstream and downstream basins for every basin that holds a reservoir

In [535]:
# - - - - - - SET UP INPUTS - - - - - - #
# HYDROBASIN 6 (smaller watersheds, match Aqueduct data)
# List of all hybas6 basins that the P4Water reservoirs lie in 
all_hy6_bas = sorted(list(set(chain.from_iterable(res_ids.HYBAS_ID.tolist()))))
all_hy6_bas = [int(x) for x in all_hy6_bas] # Make sure they are integers not floats

# FAO SUB BASINS (larger basins, named)
# Focusing on the FAO basins for now to match reservoirs to upstream and downstream basins
# List of all maj basins that the P4Water reservoirs lie in 
all_maj_bas = sorted(list(set(chain.from_iterable(res_ids.MAJ_BAS.tolist()))))
# List of all sub basins that the P4Water reservoirs lie in 
all_sub_bas = sorted(list(set(chain.from_iterable(res_ids.SUB_BAS.tolist()))))
# Filter FAO data by P4W's major basins (not all subbas IDs are unique)
fao_p4w = fao[fao.MAJ_BAS.isin(all_maj_bas)]

# - - - - - - FIND HYDRO CONNECTIONS - - - - - - #
# Find upstream and downstream basins for every basin that contains a reservoir
df_hy6_hydro = basin_upstream_downstream(all_hy6_bas, hy6, "HYBAS_ID", "NEXT_DOWN")
df_fao_hydro = basin_upstream_downstream(all_sub_bas, fao_p4w, "SUB_BAS", "TO_BAS")

# - - - - - - CLEAN DATA - - - - - - #
# Clean up data.
# FAO: Remove non-basins (-999) from downstream lists
df_fao_hydro['Downstream'] = df_fao_hydro['Downstream'].apply(lambda x: [i for i in x if i != -999])

# Hybas6: convert HYBAS_ID to pFAF_IDs
df_pf6_hydro = df_hy6_hydro.reset_index()
df_pf6_hydro['Upstream'] = df_pf6_hydro['Upstream'].apply(lambda x: hy6.PFAF_ID[hy6.HYBAS_ID.isin(x)].tolist())
df_pf6_hydro['Downstream'] = df_pf6_hydro['Downstream'].apply(lambda x: hy6.PFAF_ID[hy6.HYBAS_ID.isin(x)].tolist())
df_pf6_hydro['index'] = df_pf6_hydro['index'].apply(lambda x: hy6.PFAF_ID[hy6.HYBAS_ID == x].values[0])
df_pf6_hydro.set_index(['index'], inplace = True)

check: 1


In [536]:
# Function to match reservoirs to upstream and downstream basins
def reservoir_upstream_downstream(with_bas, hydro, hydro_type):
    # Create blank lists to hold up and down stream basins
    hydro_connections = []
    # For every subbasin containing the reservoir:
    for w in with_bas:
        # FInd its corresponding upstream and downstream basins
        hydro_connections.append(hydro.loc[w, hydro_type])
      
    # Once all up / down stream basins are found, clean up list (unnest, remove duplicates)
    hydro_clean = sorted(list(set(chain.from_iterable(hydro_connections))))
    return hydro_clean

In [537]:
# Create columns to hold upstream and downstream basins per watershed type
res_ids['PFAF_UP'] = np.nan # Hydrobasin 6
res_ids['PFAF_DN'] = np.nan # Hydrobasin 6
res_ids['SUB_UP'] = np.nan # FAO
res_ids['SUB_DN'] = np.nan # FAO

# Reservoirs to PFAFIDs
res_ids['PFAF_UP'] = res_ids['PFAF_ID'].apply(lambda x: reservoir_upstream_downstream(x, df_pf6_hydro, 'Upstream'))
res_ids['PFAF_DN'] = res_ids['PFAF_ID'].apply(lambda x: reservoir_upstream_downstream(x, df_pf6_hydro, 'Downstream'))
# Reservoirs to FAO
res_ids['SUB_UP'] = res_ids['SUB_BAS'].apply(lambda x: reservoir_upstream_downstream(x, df_fao_hydro, 'Upstream'))
res_ids['SUB_DN'] = res_ids['SUB_BAS'].apply(lambda x: reservoir_upstream_downstream(x, df_fao_hydro, 'Downstream'))

In [574]:
# Create shapefile with just subbasins for P4W
withs = sorted(list(set(chain.from_iterable(res_ids.SUB_BAS.tolist()))))
ups = sorted(list(set(chain.from_iterable(res_ids.SUB_UP.tolist()))))
downs = sorted(list(set(chain.from_iterable(res_ids.SUB_DN.tolist()))))
all_sub_bas = sorted(list(set(withs + ups + downs)))
p4w_fao = fao[(fao.MAJ_BAS.isin(all_maj_bas)) & (fao.SUB_BAS.isin(all_sub_bas))]
p4w_fao.to_file(driver = 'ESRI Shapefile', filename = p4w_fao_path)

In [573]:
# Create shapefile with just hybas6 for P4W
withs = sorted(list(set(chain.from_iterable(res_ids.PFAF_ID.tolist()))))
ups = sorted(list(set(chain.from_iterable(res_ids.PFAF_UP.tolist()))))
downs = sorted(list(set(chain.from_iterable(res_ids.PFAF_DN.tolist()))))
all_hy6_bas = sorted(list(set(withs + ups + downs)))
p4w_hy6 = hy6[(hy6.PFAF_ID.isin(all_hy6_bas))]
p4w_hy6.to_file(driver = 'ESRI Shapefile', filename = p4w_hy6_path)

In [568]:
p4w_hy6

,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER_,SORT,Shape_Length,Shape_Area,geometry
2389,4.060025e+09,0.000000e+00,4.060025e+09,4.060025e+09,0.0,0.0,176.0,1574223.9,452100,0,0,1,1129.0,0.754639,0.015484,"MULTIPOLYGON (((90.63333 23.22083, 90.62917 23..."
2391,4.060975e+09,4.060025e+09,4.060025e+09,4.060025e+09,7.0,7.0,4180.4,1488646.5,452300,0,0,1,1131.0,5.067991,0.368859,"MULTIPOLYGON (((89.63333 23.54583, 89.63270 23..."
2395,4.060960e+09,4.060975e+09,4.060025e+09,4.060025e+09,123.0,123.0,3031.0,944594.9,452411,0,0,2,1135.0,4.850206,0.268521,"MULTIPOLYGON (((89.60000 23.94583, 89.60057 23..."
2400,4.060940e+09,4.060960e+09,4.060025e+09,4.060025e+09,320.5,320.5,1153.5,929923.1,452413,0,0,2,1140.0,2.163145,0.102686,"MULTIPOLYGON (((88.02083 24.88750, 88.02304 24..."
2405,4.060930e+09,4.060940e+09,4.060025e+09,4.060025e+09,394.6,394.6,670.9,927193.7,452415,0,0,2,1145.0,1.585489,0.059865,"MULTIPOLYGON (((87.94167 24.82083, 87.93750 24..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2760,4.060988e+09,4.061001e+09,4.060032e+09,4.060032e+09,663.3,663.3,15501.8,38481.7,454065,0,0,1,1500.0,6.485675,1.361001,"MULTIPOLYGON (((79.48333 22.54583, 79.47500 22..."
2761,4.060980e+09,4.060988e+09,4.060032e+09,4.060032e+09,883.1,883.1,9247.9,18364.7,454067,0,0,1,1501.0,7.213291,0.810438,"MULTIPOLYGON (((80.90833 22.11667, 80.91577 22..."
2762,4.060980e+09,4.060988e+09,4.060032e+09,4.060032e+09,883.1,883.1,4614.8,4614.9,454066,0,0,2,1502.0,3.924973,0.406687,"MULTIPOLYGON (((79.82500 23.16667, 79.81250 23..."
2763,4.060990e+09,4.060980e+09,4.060032e+09,4.060032e+09,1101.5,1101.5,4214.2,4214.2,454068,0,0,2,1503.0,3.614234,0.368936,"MULTIPOLYGON (((80.90833 22.11667, 80.88831 22..."


<h3>COLUMN NAMES</h3>


<li>CWC_NAME = Name used by CWC daily water levels data</li>
<li>PFAF_ID = Unique watershed identifier for Hydrobasin 6 system (used by Aqueduct) </li>
<li>HYBAS_ID = Additional watershed identifier for Hydrobasin 6 system (only used to ID upstream/downstream)  </li>
<li>GID_2 = Admin Level 2 ID (one level below states/provinces) </li>
<li>MAJ_BAS = FAO Major Basins IDs. These can be matched to basin names </li>
<li>SUB_BAS = FAO Sub Basins IDs. These can be matched to basin names  </li>
<li>PFAF_UP = PFAF IDs for basins Upstream of reservoir </li>
<li>PFAF_DN = PFAF IDs for basins Downstream of reservoir  </li>
<li>SUB_UP = FAO Sub Basin IDs for basins Upstream of reservoir  </li>
<li>SUB_DN = FAO Sub Basin IDs for basins Downstream of reservoir  </li>

In [527]:
res_ids.to_csv(res_ids_path)
res_ids

,CWC_NAME,PFAF_ID,HYBAS_ID,GID_2,MAJ_BAS,SUB_BAS,PFAF_UP,PFAF_DN,SUB_UP,SUB_DN
GRAND_ID,,,,,,,,,,
4857,Tilaiya,[453146],[4060963930.0],[IND.15.10_1],[5036],[36002],[],"[453141, 453143, 453145]",[],[]
4858,Rihand,[452438],[4060903120.0],"[IND.19.46_1, IND.34.72_1]",[5035],[35031],[],"[452100, 452300, 452411, 452413, 452415, 45241...",[35030],"[35029, 35032, 35034, 35046, 35048, 35084]"
4862,Maithon,[453146],[4060963930.0],"[IND.36.3_1, IND.15.11_1, IND.15.4_1]",[5036],[36002],[],"[453141, 453143, 453145]",[],[]
4863,Tenughat,[453149],[4060961650.0],"[IND.15.20_1, IND.15.1_1]",[5036],[36002],[],"[453141, 453143, 453145, 453147]",[],[]
4865,Panchet,[453147],[4060964080.0],"[IND.36.18_1, IND.15.4_1]",[5036],[36002],"[453148, 453149]","[453141, 453143, 453145]",[],[]
4881,Bargi,[454067],[4060979850.0],"[IND.19.40_1, IND.19.27_1, IND.19.24_1]",[5048],[48002],"[454068, 454069]","[454061, 454063, 454065]",[48001],"[48004, 48005, 48006, 48007, 48008, 48009, 480..."
4898,Hirakud,"[453230, 453250, 453240]","[4061031590.0, 4061017990.0, 4061017890.0]","[IND.26.4_1, IND.26.28_1, IND.26.14_1]","[5038, 5038, 5038]","[38005, 38006, 38007]","[453240, 453250, 453260, 453270, 453280, 45329...","[453210, 453230]","[38001, 38002, 38003, 38004, 38005, 38006]","[38007, 38011, 38012]"
4946,Sriramsagar,"[453479, 453477, 453478]","[4061060950.0, 4061061290.0, 4061060870.0]","[IND.32.8_1, IND.32.1_1]",[5040],[40010],"[453478, 453479, 453481, 453482, 453483, 45348...","[453410, 453430, 453450, 453471, 453473, 45347...","[40001, 40002, 40003, 40004, 40005, 40006, 400...","[40013, 40020, 40023]"
5014,Mettur,[453805],[4061145690.0],"[IND.31.18_1, IND.31.5_1, IND.16.8_1]",[5044],[44005],"[453806, 453807, 453808, 453809]","[453801, 453803]","[44001, 44002, 44003, 44004]",[44007]


# TO COME: Match Powerplants to reservoirs. Confirm link using Vasudha 

In [579]:
# Read in P4W Hybas 6's
p4why6 = gpd.read_file(context_gdb, layer='P4W_hy6')

In [584]:
pp = gpd.read_file(context_gdb, layer='PowerPlant')
pp_bas = gpd.sjoin(pp, p4why6.filter(['PFAF_ID', 'geometry']), how="left", op='intersects') # Join shapefile to reservoir shapefile



In [587]:
pp_bas.to_csv('power_plants_lookup.csv')

In [581]:
pp_bas

,FID_1,country,country_lo,name,gppd_idnr,capacity_m,latitude,longitude,fuel1,fuel2,...,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER_,SORT,Shape_Leng,Shape_Length,Shape_Area
0,11363,IND,India,ANPARA C TPS,IND0000013,2630.0,24.2010,82.7891,Coal,Oil,...,68436.5,68436.5,452438.0,0.0,0.0,3.0,1186.0,21.954125,21.954125,6.050292
1,12123,IND,India,JOJBERA,IND0000160,427.5,22.7554,86.2491,Coal,Oil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12264,IND,India,SINGRAULI STPS,IND0000415,2000.0,24.1033,82.7068,Coal,Oil,...,68436.5,68436.5,452438.0,0.0,0.0,3.0,1186.0,21.954125,21.954125,6.050292
3,12274,IND,India,STERLITE TPP,IND0000426,2400.0,21.8144,84.0404,Coal,Oil,...,12700.8,12700.8,453240.0,0.0,0.0,2.0,1335.0,7.195533,7.195533,1.109666
4,12328,IND,India,TENUGHAT,IND0000452,420.0,23.7573,85.8936,Coal,Oil,...,4636.8,4636.8,453149.0,0.0,0.0,1.0,1316.0,3.947647,3.947647,0.409565
5,12589,IND,India,RAGHUNATHPUR TPP PH-I,IND0000355,1200.0,23.6220,86.6610,Coal,Oil,...,4724.8,11346.5,453147.0,0.0,0.0,1.0,1314.0,3.815300,3.815300,0.417315
6,12803,IND,India,RIHAND,IND0000375,3000.0,24.0270,82.7915,Coal,Oil,...,68436.5,68436.5,452438.0,0.0,0.0,3.0,1186.0,21.954125,21.954125,6.050292
7,12816,IND,India,Ramagundam Solar Power Plant,WRI1026192,10.0,18.7640,79.4873,Solar,,...,4985.1,102647.4,453473.0,0.0,0.0,1.0,1366.0,3.800415,3.800415,0.426112
8,12845,IND,India,SANTALDIH,IND0000393,500.0,23.6013,86.4666,Coal,Oil,...,4724.8,11346.5,453147.0,0.0,0.0,1.0,1314.0,3.815300,3.815300,0.417315
9,12851,IND,India,SEIONI TPP,IND0000397,600.0,22.7355,79.9123,Coal,Oil,...,9247.9,18364.7,454067.0,0.0,0.0,1.0,1501.0,7.213291,7.213291,0.810438
